In [1]:
import tensorflow as tf
from collections import namedtuple

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, [1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


In [3]:
def build_model(conv1_depth=32, conv2_depth=64, fc_width=1024, lr=1e-4):
    
    tf.reset_default_graph()
    
    
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
        x_image = tf.reshape(x, [-1,28,28,1], name='x_image')

    with tf.name_scope('targets'):
        y_ = tf.placeholder(tf.float32, shape=[None, 10], name='y_')


    with tf.name_scope('conv1_layer'):
        W_conv1 = weight_variable([5, 5, 1, conv1_depth], 'W_conv1')
        b_conv1 = bias_variable([conv1_depth], 'b_conv1')
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)
        tf.summary.histogram('W_conv1', W_conv1)
        tf.summary.histogram('b_conv1', b_conv1)
        

    with tf.name_scope('conv2_layer'):
        W_conv2 = weight_variable([5, 5, conv1_depth, conv2_depth], 'W_conv2')
        b_conv2 = bias_variable([conv2_depth], 'b_conv2')
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)
        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*conv2_depth])

        tf.summary.histogram('W_conv2', W_conv2)
        tf.summary.histogram('b_conv2', b_conv2)
        
    with tf.name_scope('fc_layer'):
        W_fc1 = weight_variable([7 * 7 * conv2_depth, fc_width], 'W_fc')
        b_fc1 = bias_variable([fc_width], 'b_fc')
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

        tf.summary.histogram('W_fc1', W_fc1)
        tf.summary.histogram('b_fc1', b_fc1)
        
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    with tf.name_scope('logits'):
        W_fc2 = weight_variable([fc_width, 10], 'W_logits')
        b_fc2 = bias_variable([10], 'b_logits')
        logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

        tf.summary.histogram('W_fc2', W_fc2)
        tf.summary.histogram('b_fc2', b_fc2)
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits, name='loss')
        cost = tf.reduce_mean(loss, name='cost')
        tf.summary.scalar('cost', cost)

    with tf.name_scope('train'):
        opt = tf.train.AdamOptimizer(lr).minimize(cost)

    with tf.name_scope('preds'):
        pred_prob = tf.nn.softmax(logits)
        tf.summary.histogram('pred_prob', pred_prob)
        preds = tf.argmax(logits,1)
        correct_prediction = tf.equal(preds, tf.argmax(y_,1), name='prediction')
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
        tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all()
    
    # Export the nodes 
    export_nodes = ['x', 'y_', 'accuracy', 'keep_prob', 'merged', 'opt']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

In [4]:

def train(model, train_writer, test_writer):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for i in range(10000):
            batch = mnist.train.next_batch(25) #halved the batch size to allow larger models to train
            if i%25 == 0:
            #print(batch[0])
                train_accuracy, summary = sess.run([model.accuracy, model.merged], feed_dict={
                    model.x: batch[0], 
                    model.y_: batch[1], 
                    model.keep_prob: 1.0
                })
                #print("step %d, training accuracy %g"%(i, train_accuracy))
                train_writer.add_summary(summary, i)

            sess.run([model.opt], feed_dict={
                        model.x: batch[0], 
                        model.y_: batch[1], 
                        model.keep_prob: 0.5})

            if (i%100 == 0):
                acc, summary = sess.run([model.accuracy, model.merged], feed_dict={
                        model.x: mnist.test.images, 
                        model.y_: mnist.test.labels, 
                        model.keep_prob: 1.0})
                #print("test accuracy %g"%acc)
                test_writer.add_summary(summary, i)

            if i%500 == 0:
                print('on iteration: ', i, 'train acc', train_accuracy, 'test_acc', acc)


In [5]:
for fc_width in [256, 512, 1024]:
    for lr in [1e-2, 1e-4]:
        for conv1_d, conv2_d in [(8, 16), (32, 64)]:
            
            train_log_string = './logs/params/train,fc_w={},lr={},conv1={},conv2={}'.format(fc_width, lr, conv1_d, conv2_d)
            test_log_string = './logs/params/test,fc_w={},lr={},conv1={},conv2={}'.format(fc_width, lr, conv1_d, conv2_d)
            print('Starting File', train_log_string)
            try:
                model = build_model(conv1_depth=conv1_d, conv2_depth=conv2_d, fc_width=fc_width, lr=lr)
                train_writer = tf.summary.FileWriter(train_log_string)
                test_writer = tf.summary.FileWriter(test_log_string)
                train(model, train_writer, test_writer)
            except Exception as e:
                print('Failed', train_log_string)
                print(type(e), e.args)
            



Starting File ./logs/params/train,fc_w=256,lr=0.01,conv1=8,conv2=16
on iteration:  0 train acc 0.04 test_acc 0.127
on iteration:  500 train acc 0.96 test_acc 0.927
on iteration:  1000 train acc 1.0 test_acc 0.9462
on iteration:  1500 train acc 0.92 test_acc 0.9433
on iteration:  2000 train acc 1.0 test_acc 0.9648
on iteration:  2500 train acc 1.0 test_acc 0.9552
on iteration:  3000 train acc 0.96 test_acc 0.9505
on iteration:  3500 train acc 0.88 test_acc 0.9588
on iteration:  4000 train acc 0.96 test_acc 0.9533
on iteration:  4500 train acc 0.92 test_acc 0.9526
on iteration:  5000 train acc 1.0 test_acc 0.9603
on iteration:  5500 train acc 0.96 test_acc 0.9467
on iteration:  6000 train acc 0.96 test_acc 0.9477
on iteration:  6500 train acc 0.88 test_acc 0.9616
on iteration:  7000 train acc 1.0 test_acc 0.9609
on iteration:  7500 train acc 0.92 test_acc 0.9564
on iteration:  8000 train acc 1.0 test_acc 0.962
on iteration:  8500 train acc 0.92 test_acc 0.9418
on iteration:  9000 train a